In [1]:
import random
import operator
import json

In [2]:
from Hola_Dictionary import Hola_Dictionary_English, Hola_Dictionary_Stop, Hola_Dictionary
from Hola_Dictionary_Statistics import Hola_Dictionary_Markov_Chain, Hola_Dictionary_Statistics
from Hola_Test_System import Hola_Test_System

Загрузим слова, не входящие в словарь

In [3]:
eng_dict = Hola_Dictionary_English()
eng_dict.read('words.txt')

stop_dict = Hola_Dictionary_English()
stop_dict.read('stop.txt')

In [4]:
print len(eng_dict.get_words())
print len(stop_dict.get_words())

630404
654431


In [5]:
eng_chain = Hola_Dictionary_Markov_Chain(eng_dict, order=3)
eng_chain.train(0.99)

In [6]:
eng_chain.save_to_file('mc_eng_3_99.json')

In [13]:
print len(eng_chain._chain)

21952


In [7]:
stop_chain = Hola_Dictionary_Markov_Chain(stop_dict, order=3)
stop_chain.train(0.99)

In [8]:
stop_chain.save_to_file('mc_stop_3_99.json')

In [15]:
class Hola_Dictionary_Rare(Hola_Dictionary_English):
    _stat = None
    _filtered = {}
    
    def get_rare_words(self, percentile):
        if self._stat is None:
            self._stat = Hola_Dictionary_Statistics(self)
        if self._filtered.get(percentile) is not None:
            return self._filtered[percentile]
        signed_len = self._stat.get_significant_length(percentile)
        filtered = []
        for word in self.get_words():
            if len(word) not in signed_len:
                filtered.append(word)
        self._filtered[percentile] = filtered
        return filtered
    
    def save_to_file(self, file_name, percentile):
        rare_words = self.get_rare_words(percentile)
        f = open(file_name, 'w+')
        f.write(json.dumps(rare_words))
        f.close()

In [16]:
rare_dict = Hola_Dictionary_Rare()
rare_dict.read('words.txt')
#rare_dict.save_to_file('rare_95.json', 0.95)
#rare_dict.save_to_file('rare_99.json', 0.99)

Начинаем подбирать параметры модели

In [9]:
test_system = Hola_Test_System('test/cases/')     

In [18]:
rare_words = rare_dict.get_rare_words(0.99)
stat = Hola_Dictionary_Statistics(eng_dict)
signed_len = stat.get_significant_length(0.99)
print sorted(signed_len)

[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]


In [11]:
def run_test(param, count, LOG):
    random.seed(48327104)
    test_cases_start = random.randint(1000000000, 9000000000)
    test_cases_count = count
    test_case_numbers = range(test_cases_start, test_cases_start + test_cases_count)

    correct = 0
    tf = 0
    ft = 0
    total = 0
    min_list = []


    for test_cases_number in test_case_numbers:
        test_case = test_system.load_test_case(test_cases_number)
        for word_with_suffix in test_case:
            word = word_with_suffix#.replace("'s", '')
            total += 1
            p_eng_list = eng_chain.test(word)
            p_stop_list = stop_chain.test(word)
        
            prod_eng = reduce(operator.mul, [p for p in p_eng_list], 1)
            prod_stop = reduce(operator.mul, [p for p in p_stop_list], 1)
            if prod_eng == 0:
                result = False
            elif prod_eng > prod_stop:
                result = True
                if len(word) < 5:
                    result = False
                diff = [(s/e) for s, e in zip(p_stop_list, p_eng_list)]
                if max(diff) > 8:
                    result = False
                if len(word) > 13:
                    result = False
            else:
                result = False
                
                
            #if len(word) not in signed_len:
            #    result = word in rare_words
            
            if result == test_case[word_with_suffix]:
                correct += 1
                
            else:
                
                if result:
                    #print word
                    #print ["%.2f" % p for p in p_eng_list]
                    #print ["%.2f" % p for p in p_stop_list]
                    tf += 1
                else:                    
                    ft += 1
                    
    correct = correct / float(total)
    tf = tf / float(total)
    ft = ft / float(total)
    return correct, tf, ft

In [12]:
import numpy as np

max_correct = 0
max_correct_param = None
for param in np.arange(9, 10, 1):
    correct, tf, ft = run_test(param, 200, 1)
    print param, ': ', correct, tf, ft
    if correct > max_correct:
        max_correct = correct
        max_correct_param = param
        
print max_correct, max_correct_param

9 :  0.65155 0.2571 0.09135
0.65155 9
